In [5]:
#get the facebook module
import facebook
import urllib2
import urllib
import bs4
import json
from PIL import Image


In [5]:
#create access token and get the graphAPI object
#You need to go to https://developers.facebook.com/tools/explorer and create a token for yourself.
#Make sure you fill up all the checkpoints while requesting for a token
access_token_got = '<My Token>'
graph = facebook.GraphAPI(access_token=access_token_got,version=2.7)

In [ ]:
'''
The results page on a particular query looks like this.

{
  "data": [
    {
      "id": "Some ID",
      "name": "Some Name"
    },
    {
      "id": "Some ID",
      "name": "Some Name"
    },
    ....
    
  ],
  "paging": {
    "cursors": {
      "before": "Some ID",
      "after": "Some ID"
    },
    "next": "next Link"
}
 
It contains finite number of results so you basically need to go the next link for another set
The next set of functions basically perform this work for us
'''


In [16]:
#this is the result of the first page. The results have a name and id and then a next link . 
# I store the  ids as a list and i really don't care about the name and then move on to the next link till the next link is not null

def store_data(data):
    dictionary_ids = []
    for chunks in data:
        dictionary_ids.append(chunks['id'])
    return dictionary_ids


In [3]:
#this functions searches for the next link on the results page
def get_next_link(data):
    if data.has_key('next'):
        return data['next']
    return None

In [23]:
#this is the result of the first page. The results have a name and id and then a next link . 
# I store the  ids as a list and i really don't care about the name and then move on to the next link till the next link is not null

def store_data(data):
    dictionary_ids = []
    for chunks in data:
        dictionary_ids.append(chunks['id'])
    return dictionary_ids



In [2]:
#this gets the next link from each result data object
def get_next_link(data):
    if data.has_key('next'):
        return data['next']
    return None

In [6]:
#this pulls of the results from the link that is recieved
def get_next_data_object(link):
    req = urllib2.Request(link, headers={'User-Agent' : "Magic Browser"}) 
    source = urllib2.urlopen( req )
    json_source = json.loads(source.read())
    return json_source

In [1]:
#this function uses all the above function to create a comprehensive list of ids for querying profile pictures
def create_list(page):
    main_id_list = []
    while(get_next_link(page['paging'])):
        data_recieved = (store_data(data=page['data']))
        main_id_list.extend(data_recieved)
        page = get_next_data_object(get_next_link(page['paging']))
    return main_id_list
        

In [4]:
#this uses the ids to get the profile pictures()
def get_pictures(id_list,graph_object):
    for _id in id_list:
        request_link_for_picture = str(_id)+"/picture?height=1000&width=1000"
        data = graph_object.request(request_link_for_picture)
        link_for_picture = data['url']
        urllib.urlretrieve(link_for_picture,str(_id)+".jpg")
        

In [ ]:
#Although i have not tested it with colored images yet, I was quite apprehensive that averaging the image with colors will
#produce nice results  :P
def convert_to_grayscale(id_list):
    for ids in id_list:
        name = str(ids)+".jpg"
        img = Image.open(name).convert('L')
        img.save(str(ids)+"_gray.jpg");

In [17]:
#now get the post id . I just went to my latest profile picture upload and found the id on the link
first_page = graph.request('1342144572536371/likes')
final_ids = create_list(first_page)
get_pictures(final_ids,graph)
convert_to_grayscale(final_ids)